# The Marmousi elastic model

Let's make a synthetic dataset and write it out to a SEG-Y file.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from obspy.io.segy.segy import _read_segy

In [ ]:
ls -l ../data/marmousi

In [ ]:
!cat ../data/marmousi/README

<hr />

We're going to make our own synthetic.

## Read the elastic data

In [ ]:
velocity = _read_segy('../data/marmousi/velocity.segy')
density = _read_segy('../data/marmousi/density.segy')

In [ ]:
velocity.binary_file_header

In [ ]:
s = velocity.textual_file_header.decode()

print('\n'.join(map(''.join, zip(*[iter(s)]*80))))

In [ ]:
def get_data(section):
    nsamples = section.binary_file_header.number_of_samples_per_data_trace
    dt = section.binary_file_header.sample_interval_in_microseconds 

    ntraces = len(section.traces)  # Get from actual file contents.
    tbase = np.arange(0, nsamples * dt / 1000.0, dt)
    vol = np.stack(t.data for t in section.traces)
    ma, mi = np.amax(vol), np.amin(vol)

    print('number of traces (ntraces)   :', ntraces)
    print('samples per trace (nsamples) :', nsamples)
    print('sample interval              :', dt)
    print('data range                   : {} - {}'.format(mi, ma))

    return tbase, vol

In [ ]:
t, vp = get_data(velocity)

In [ ]:
_, rho = get_data(density)

## Deal with weirdness

The data are transposed: the so-called traces in the ObsPy stream are really timeslices, not traces. Very strange. We'll fix it.

In [ ]:
vp.shape

In [ ]:
vp = vp.T
rho = rho.T

nsamples = 751
t = np.linspace(0, (nsamples-1)*0.004, nsamples)

In [ ]:
vp.shape

Now the data are organized correctly. We have to transpose them to display them, but the first dimension is traces.

In [ ]:
fig = plt.figure(figsize = (20,20))

ax1 = fig.add_subplot(121)
ax1.set_title(r'$V_\mathrm{P}\ \ \mathrm{[m/s]}$')
img = ax1.imshow(vp.T, aspect=2, cmap='viridis')
plt.colorbar(img, shrink=0.2)

ax2 = fig.add_subplot(122)
ax2.set_title(r'$\rho\ \ \mathrm{[kg/m^3]}$')
img = ax2.imshow(rho.T, aspect=2, cmap='viridis')
plt.colorbar(img, shrink=0.2)

plt.show()

## Make a synthetic

In [ ]:
imp = vp * rho

def make_rc(imp):
    upper = imp[:,  :-1]
    lower = imp[:, 1:  ]
    
    return (lower - upper) / (lower + upper)

rc = make_rc(imp)

plt.imshow(rc.T, aspect=2, cmap="Greys")
plt.show()

In [ ]:
from scipy.signal import ricker

f = 25
wavelet = ricker(100, 1000/(4*f))

plt.plot(wavelet)
plt.show()

In [ ]:
def convolve(trace):
    return np.convolve(trace, wavelet, mode='same')

synth = np.apply_along_axis(convolve,
                            axis=-1,
                            arr=rc)

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(synth.T, cmap="Greys", aspect=2)
plt.colorbar()
plt.show()

In [ ]:
np.amin(synth), np.amax(synth)

In [ ]:
synth.dtype

## Write a SEG-Y file

In [ ]:
from obspy.core import Trace, Stream, UTCDateTime
from obspy.io.segy.segy import SEGYTraceHeader
from obspy.core import AttribDict
from obspy.io.segy.segy import SEGYBinaryFileHeader

If you're on a 64-bit machine and Python environment, the data are currently 64-bit — 8 bytes per sample. We don't want or need this many bytes, so we'll reduce the bit-depth to 32-bit:

In [ ]:
synth = synth.astype(np.float32)
synth.dtype

In [ ]:
stream = Stream()

# Add the traces to the stream.
for i, trace in enumerate(synth):

    # Make the trace.
    tr = Trace(trace)

    # Add required data.
    tr.stats.delta = 0.004
    tr.stats.starttime = 0  # Not strictly required.

    # Add yet more to the header (optional).
    tr.stats.segy = {'trace_header': SEGYTraceHeader()}
    tr.stats.segy.trace_header.trace_sequence_number_within_line = i + 1
    tr.stats.segy.trace_header.receiver_group_elevation = 0

    # Append the trace to the stream.
    stream.append(tr)
    
# Add a text header.
stream.stats = AttribDict()
stream.stats.textual_file_header = '{:80s}'.format('This is the textual header.').encode()
stream.stats.textual_file_header += '{:80s}'.format('This file contains seismic data.').encode()

# Add a binary header.
stream.stats.binary_file_header = SEGYBinaryFileHeader()
stream.stats.binary_file_header.trace_sorting_code = 4
stream.stats.binary_file_header.seg_y_format_revision_number = 0x0100

In [ ]:
import sys

stream.write('../data/marmousi/synthetic_25Hz.sgy',
             format='SEGY',
             data_encoding=5,
             byteorder=sys.byteorder)

<hr />

© Agile Scientific 2017